## Метрики качества

In [19]:
import pandas as pd
import numpy as np
import sklearn.metrics as mt

> Загрузите файл classification.csv. В нем записаны истинные классы объектов выборки (колонка true) и ответы некоторого классификатора (колонка predicted).

> Подсчитайте величины TP, FP, FN и TN согласно их определениям. Например, FP — это количество объектов, имеющих класс 0, но отнесенных алгоритмом к классу 1. Ответ в данном вопросе — четыре числа через пробел.

In [52]:
cln = pd.read_csv("classification.csv")
errors = {"TP":0, "TN":0, "FP":0, "FN":0}
def is_TP(x):
    return (x[0], x[1]) == (1, 1)
def is_FN(x):
    return (x[0], x[1]) == (1, 0)
def is_FP(x):
    return (x[0], x[1]) == (0, 1)
def is_TN(x):
    return (x[0], x[1]) == (0, 0)
cln["is_TP"] = cln.apply(is_TP, axis=1)
cln["is_FN"] = cln.apply(is_FN, axis=1)
cln["is_FP"] = cln.apply(is_FP, axis=1)
cln["is_TN"] = cln.apply(is_TN, axis=1)

TP, FN, FP, TN = cln[["is_TP", "is_FN", "is_FP", "is_TN"]].sum()
print(TP, FP, FN, TN)

43 34 59 64


> Посчитайте основные метрики качества классификатора:
>
>Accuracy (доля верно угаданных) — sklearn.metrics.accuracy_score
Precision (точность) — sklearn.metrics.precision_score
Recall (полнота) — sklearn.metrics.recall_score
F-мера — sklearn.metrics.f1_score
В качестве ответа укажите эти четыре числа через пробел.

In [53]:
accuracy = mt.accuracy_score(y_pred=cln["pred"], y_true=cln["true"])
precision = mt.precision_score(y_pred=cln["pred"], y_true=cln["true"])
recall = mt.recall_score(y_pred=cln["pred"], y_true=cln["true"])
f1_score = mt.f1_score(y_pred=cln["pred"], y_true=cln["true"])
print(accuracy, precision, recall, f1_score)

0.535 0.558441558442 0.421568627451 0.480446927374


> Имеется четыре обученных классификатора. В файле scores.csv записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:

> * для логистической регрессии — вероятность положительного класса (колонка score_logreg),
> * для SVM — отступ от разделяющей поверхности (колонка score_svm),
> * для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn),
> * для решающего дерева — доля положительных объектов в листе (колонка score_tree).

In [30]:
scores = pd.read_csv("scores.csv")
scores.head()

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333
2,0,0.382315,-0.245701,0.000000,0.000000
3,1,0.506797,-0.137058,0.000000,0.105263
4,1,0.488781,-0.154148,0.000000,0.105263


### Хозяйке на заметку

ROC - метрику можно считать не только по вероятностям принадлежности, но и по степеням уверенности

> _score : array, shape = [n_samples] or [n_samples, n_classes]
> Target scores, can either be probability estimates of the positive class, confidence values, or binary decisions.

[Источник](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)

In [33]:
S_logreg = mt.roc_auc_score(y_true=scores["true"], y_score=scores["score_logreg"])
S_svm = mt.roc_auc_score(y_true=scores["true"], y_score=scores["score_svm"])
S_knn = mt.roc_auc_score(y_true=scores["true"], y_score=scores["score_knn"])
S_tree = mt.roc_auc_score(y_true=scores["true"], y_score=scores["score_tree"])

print(S_logreg, S_svm, S_knn, S_tree)

0.71918767507 0.708683473389 0.635154061625 0.691926770708


> Какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70% ? Какое значение точности при этом получается?

> Чтобы получить ответ на этот вопрос, найдите все точки precision-recall-кривой с помощью функции sklearn.metrics.precision_recall_curve. Она возвращает три массива: precision, recall, thresholds. В них записаны точность и полнота при определенных порогах, указанных в массиве thresholds. Найдите максимальной значение точности среди тех записей, для которых полнота не меньше, чем 0.7.

In [41]:
pts_logreg = mt.precision_recall_curve(scores["true"], scores["score_logreg"])
pts_svm = mt.precision_recall_curve(scores["true"], scores["score_svm"])
pts_knn = mt.precision_recall_curve(scores["true"], scores["score_knn"])
pts_tree = mt.precision_recall_curve(scores["true"], scores["score_tree"])
type(pts_logreg[2])

numpy.ndarray

In [54]:
logreg_max_prec = pts_logreg[0][(pts_logreg[1] > 0.7)].max()
svm_max_prec = pts_svm[0][(pts_svm[1] > 0.7)].max()
knn_max_prec = pts_knn[0][(pts_knn[1] > 0.7)].max()
tree_max_prec = pts_tree[0][(pts_tree[1] > 0.7)].max()
print(logreg_max_prec, svm_max_prec, knn_max_prec, tree_max_prec)

0.63025210084 0.622807017544 0.606557377049 0.651785714286
